In [1]:
import numpy as np
import os
import random

import config
from scenario import *
from tileset import *
from mdlstm import *

from time import time

import tensorflow.contrib.slim as slim

Нека изчетем данните. Измежду тях ще изберем само карти за двама играчи, с най-малкия възможен размер (64 X 64), и използващи плочки тип `jungle`. Oчакваме картите да са 11 на брой. Бройката е доста малка, но ще разчитаме на [неразумната ефективност на рекурентните невронни мрежи](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

In [2]:
scenarios = []
scenarios += process_scenarios(os.path.join(config.STARCRAFT_ROOT, 'Maps'))
for directory in config.MAP_DIRECTORIES:
    scenarios += process_scenarios(directory)

scenarios = [x for x in scenarios if x.alliances == x.human_players == 2 and x.tileset == Tileset.JUNGLE and x.width == x.height == 64]

len(scenarios)

11

Ще имплементираме помощни функции, които извличат feature-и от плочка. Плочките силно зависят от околните. И може би има смисъл да приложим нещо подобно на word2vec.

Но за сега ще направим малко feature engineering, за да изпробваме модела. Ще работим с 3 feature-а:
- осреднена височина на плочката
- коефициент за това върху каква част от плочката може да се ходи
- дали върху плочката може да се строи

In [3]:
h = 64
w = 64

tile_vec_size = 3
input_vec_size = 2 * tile_vec_size
output_vec_size = tile_vec_size

In [4]:
@np.vectorize
def to_features(tile):

    @np.vectorize
    def minitile_heights(minitile):
        return minitile.height

    @np.vectorize
    def minitile_walkability(minitile):
        return minitile.walkable

    return np.array([
        np.average(minitile_heights(tile.minitiles)),
        np.average(minitile_walkability(tile.minitiles)),
        tile.buildable
    ], dtype=np.float32) + 0.0001

In [5]:
def to_features_by_index(tiles, horizontal_index, vertical_index):
    if vertical_index >= 0 and horizontal_index >= 0:
        tile = tiles[vertical_index, horizontal_index]
        return to_features(tile)
    else:
        return np.zeros([tile_vec_size])

## Експлицитни feature-и на всяка стъпка

Всяка плочка ще зависи от горната и от лявата.

В нашата двуизмерна рекурентна невронна мрежа ще използваме конкатенацията на feature-ите на горната и лявата плочки за вход. За изход ще използваме feature-ите на текущата плочка. За feature-и на плочки извън игралното поле ще връщаме списък нули.

Всъщност скритият state за RNN-а също идва от горната и от лявата плочка. Така че на теория би трябвало да не е нужно да ги даваме експлицитно. Но при по-ранни експерименти имах **огромни** проблеми с числената стабилност на модела. За всякакъв смислено голям learning rate получавах NaN-ове в loss-a. Което всъщност не е изненада. Дори е описано в документацията на модела. Експлицитните feature-и са трик, който се справя с този проблем.

Експлицитните feature-и на всяка стъпка имат още един плюс. Когато използваме модела за генерация, ще можем да избираме плочка различна от най-вероятната за всяка от стъпките.

In [6]:
data = []

for scenario in scenarios:
    x = np.empty((h, w, input_vec_size), dtype=np.float32)
    y = np.empty((h, w, output_vec_size), dtype=np.float32)
    for vertical_index in range(h):
        for horizontal_index in range(w):
            top_tile_features = to_features_by_index(scenario.tiles, vertical_index - 1, horizontal_index)
            left_tile_features = to_features_by_index(scenario.tiles, vertical_index, horizontal_index - 1)
            x[vertical_index, horizontal_index, :] = np.concatenate([top_tile_features, left_tile_features])
            y[vertical_index, horizontal_index, :] = to_features_by_index(scenario.tiles, vertical_index, horizontal_index)
    data.append((x, y))

In [9]:
def batches(data, batch_size, epochs):
    all_batches = []
    for epoch in range(epochs):
        random.shuffle(data)
        all_batches += data

    for i in range(0, epochs * len(data), batch_size):
        inputs = all_batches[i: i + batch_size]
        if len(inputs) == batch_size:
            yield inputs

## Терниране на модела

Моделът идващ от модула `mdlstm`, както и кодът в тази тетрадка се базира на [tensorflow-multi-dimensional-lstm](https://github.com/philipperemy/tensorflow-multi-dimensional-lstm) написан от [Philippe Rémy](https://github.com/philipperemy) с разни промени от моя страна.

Нека на базата на този код да имплементираме примерен трениращ код и да проверим дали моделът конвергира.

In [123]:
learning_rate = 3e-3
batch_size = 6
hidden_size = 16
dtype = tf.float32

# with tf.Session() as sess:
with tf.variable_scope('fooo4', reuse=tf.AUTO_REUSE):
    x = tf.placeholder(dtype, [batch_size, h, w, input_vec_size])
    y = tf.placeholder(dtype, [batch_size, h, w, output_vec_size])

    mdrnn_while_loop = MdRnnWhileLoop(dtype)
    rnn_out, _ = mdrnn_while_loop(rnn_size=hidden_size, input_data=x)
    model_out = slim.fully_connected(inputs=rnn_out, num_outputs=output_vec_size, activation_fn=tf.nn.sigmoid)

    loss = tf.reduce_mean(tf.square(y - model_out))
    grad_update = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver(tf.global_variables())

    epochs = 500
    step = 0
    for batch in batches(data, batch_size, epochs):
        grad_step_start_time = time()

        model_preds, tot_loss_value, _ = sess.run([model_out, loss, grad_update], feed_dict={
            x: np.stack([x[0] for x in batch]),
            y: np.stack([x[1] for x in batch]),
        })

        if step % 10 == 0:
            print('steps = {0} | overall loss = {1:.5f}'.format(str(step).zfill(4), tot_loss_value))

        if tot_loss_value != tot_loss_value:
            break

        if step % 50 == 0:
            saver.save(sess, os.path.join('checkpoints', 'model'), global_step=step)

        step += 1

    print('steps = {0} | overall loss = {1:.3f}'.format(str(step).zfill(4), tot_loss_value))
    saver.save(sess, os.path.join('checkpoints', 'model'), global_step=step)

steps = 0000 | overall loss = 0.24446
steps = 0010 | overall loss = 0.18422
steps = 0020 | overall loss = 0.15899
steps = 0030 | overall loss = 0.16026
steps = 0040 | overall loss = 0.14555
steps = 0050 | overall loss = 0.14163
steps = 0060 | overall loss = 0.14052
steps = 0070 | overall loss = 0.13060
steps = 0080 | overall loss = 0.11840
steps = 0090 | overall loss = 0.10667
steps = 0100 | overall loss = 0.09992
steps = 0110 | overall loss = 0.09374
steps = 0120 | overall loss = 0.08738
steps = 0130 | overall loss = 0.08464
steps = 0140 | overall loss = 0.07915
steps = 0150 | overall loss = 0.07158
steps = 0160 | overall loss = 0.06686
steps = 0170 | overall loss = 0.06214
steps = 0180 | overall loss = 0.05708
steps = 0190 | overall loss = 0.05526
steps = 0200 | overall loss = 0.05008
steps = 0210 | overall loss = 0.04769
steps = 0220 | overall loss = 0.04510
steps = 0230 | overall loss = 0.04351
steps = 0240 | overall loss = 0.04303
steps = 0250 | overall loss = 0.03690
steps = 0260

Моделът конвергира.

## Изследване на хиперпараметрите

По-голям batch_size и learning_rate водят до NaN в стойностите за loss.
По-малки стойности водят до по-бавно учене за всяка от итерациите.
**TODO: да го демонстрирам**

Освен това няма да даваме batch_size за x и y.
Това ще направи полученият RNN модел една идея по-гъвкав за семплиране.
Но това ще направи всяка от итерациите по-бавна.
**TODO: да го демонстрирам**

Интересен е hidden_size. По-високите му стойности водят до по-бавно учене в началото.
Но след определен брой итерации резулттите са по-добри.
Което не е изненада. И тъй като всяка итерация отнема повече време, ученето не е по-бързо като време.
**TODO: да го демонстрирам**

При предишните експерименти loss-ът не падаше много под 0.2.
Въпросът е дали можем да достигнем по-добър краен резултат с по-голям hidden_size.

In [10]:
learning_rate = 3e-3
batch_size = 6
hidden_size = 32
dtype = tf.float32

# with tf.Session() as sess:
with tf.variable_scope('fooo5', reuse=tf.AUTO_REUSE):
    x = tf.placeholder(dtype, [batch_size, h, w, input_vec_size])
    y = tf.placeholder(dtype, [batch_size, h, w, output_vec_size])

    mdrnn_while_loop = MdRnnWhileLoop(dtype)
    rnn_out, _ = mdrnn_while_loop(rnn_size=hidden_size, input_data=x)
    model_out = slim.fully_connected(inputs=rnn_out, num_outputs=output_vec_size, activation_fn=tf.nn.sigmoid)

    loss = tf.reduce_mean(tf.square(y - model_out))
    grad_update = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver(tf.global_variables())

    epochs = 500
    step = 0
    for batch in batches(data, batch_size, epochs):
        grad_step_start_time = time()

        model_preds, tot_loss_value, _ = sess.run([model_out, loss, grad_update], feed_dict={
            x: np.stack([x[0] for x in batch]),
            y: np.stack([x[1] for x in batch]),
        })

        if step % 10 == 0:
            print('steps = {0} | overall loss = {1:.5f}'.format(str(step).zfill(4), tot_loss_value))

        if tot_loss_value != tot_loss_value:
            break

        if step % 50 == 0:
            saver.save(sess, os.path.join('checkpoints', 'model'), global_step=step)

        step += 1

    print('steps = {0} | overall loss = {1:.5f}'.format(str(step).zfill(4), tot_loss_value))
    saver.save(sess, os.path.join('checkpoints', 'model'), global_step=step)

steps = 0000 | overall loss = 0.21937
steps = 0010 | overall loss = 0.15768
steps = 0020 | overall loss = 0.15014
steps = 0030 | overall loss = 0.13898
steps = 0040 | overall loss = 0.13000
steps = 0050 | overall loss = 0.12841
steps = 0060 | overall loss = 0.11938
steps = 0070 | overall loss = 0.10817
steps = 0080 | overall loss = 0.10536
steps = 0090 | overall loss = 0.10429
steps = 0100 | overall loss = 0.10201
steps = 0110 | overall loss = 0.09170
steps = 0120 | overall loss = 0.08910
steps = 0130 | overall loss = 0.08931
steps = 0140 | overall loss = 0.08021
steps = 0150 | overall loss = 0.07709
steps = 0160 | overall loss = 0.07350
steps = 0170 | overall loss = 0.07072
steps = 0180 | overall loss = 0.06903
steps = 0190 | overall loss = 0.06971
steps = 0200 | overall loss = 0.06781
steps = 0210 | overall loss = 0.06564
steps = 0220 | overall loss = 0.06172
steps = 0230 | overall loss = 0.06193
steps = 0240 | overall loss = 0.05858
steps = 0250 | overall loss = 0.05693
steps = 0260